In [1]:
%load_ext autoreload
%matplotlib qt5
import numpy as np
import Z_n_circuit_utils
from Circuits import unitary_sampler
import Circuits.circuit_evolution as evolution
from itertools import combinations
import matplotlib.pyplot as pl
import matplotlib as mpl
import os
import pickle
from colour import Color
from scipy.special import binom
from circuit_utils import max_value, get_Z_N_entropy, Z_n_max_value
%autoreload 2

In [2]:
def get_prob_data(file):
    full_data = []
    for merged_file in os.listdir(file):
        with open(file+'/'+merged_file,'rb') as f:
            data = pickle.load(f)
            full_data.extend(list(data['probs']))
    return full_data

def get_U_1_data(file):
    full_data = []
    for merged_file in os.listdir(file):
        with open(file+'/'+merged_file,'rb') as f:
            data = pickle.load(f)
            full_data.extend(list(data['U_1_entropy']))
    return full_data

def get_Z_n_data(file):
    full_data = []
    for merged_file in os.listdir(file):
        with open(file+'/'+merged_file,'rb') as f:
            data = pickle.load(f)
            if 'Z_n_entropy' in data: full_data.extend(list(data['Z_n_entropy']))
    return full_data

def get_std(file):
    entropy = []
    for merged_file in os.listdir(file):
        with open(file+'/'+merged_file,'rb') as f:
            data = pickle.load(f)
            entropy.extend(data['std'])
    return entropy

def get_L(name:str):
    L = 0
    index2 = name.index('_T')
    index1 = name.index('L=')
    L = int(name[index1+2:index2])
    return L

def get_p(name:str):
    p = 0
    index1 = name.index('p=')
    index2 = name.index('_g=')
    p = float(name[index1+2:index2])
    return p

def get_g(name:str):
    g = 0
    index1 = name.index('g=')
    g = float(name[index1+2:])
    return g

def get_T(name:str):
    T = 0
    index2 = name.index('_N')
    index1 = name.index('T=')
    T = int(name[index1+2:index2])
    return T

def get_N(name:str):
    N = 0
    index1 = name.index('N=')
    N = int(name[index1+2:])
    return N

def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

In [3]:
entropy_dic = {}
Z_n_entropy_dic = {}
depth_ratio = 5
N = 4
root_direc = 'data/phase_measurement_Z_n/U_1_fuzzy_initial/'
root_direc = 'data/unitary_Z_n/U_1_fuzzy_initial/'
for L_file in os.listdir(root_direc):
    if get_N(L_file) != N:
        continue

    if L_file == '.DS_Store':
        continue

    L = get_L(L_file)
    assert get_T(L_file) == int(depth_ratio*L), "Wrong depth ratio"
    
    print(L_file)

    entropy_dic[L] = {}
    Z_n_entropy_dic[L] = {}
    sub_dir = root_direc + '/' + L_file
    for p_file in os.listdir(sub_dir):
        if 'p=' not in p_file:
            continue
        p = get_p(p_file)
        g = get_g(p_file)

        merged_dir = sub_dir + '/' + p_file + '/' + 'merged_data'
        if not os.path.isdir(merged_dir):
            continue
        if p not in entropy_dic[L]:
            entropy_dic[L][p] = {}
            Z_n_entropy_dic[L][p] = {}

        Z_n_data = np.array(get_Z_n_data(merged_dir))
        # Z_n_data = np.empty(data.shape[:2])
        # for s in range(data.shape[0]):
        #     for t in range(data.shape[1]):
        #         Z_n_data[s,t] = get_Z_N_entropy(data[s,t],L,N)
        Z_n_entropy_dic[L][p][g] = Z_n_data

        entropy_dic[L][p][g] = np.array(get_U_1_data(merged_dir))
        # the shape of entropy_dic[L][p][g] is (num,T) where num is the number of samples, T is the time steps
    
    if len(entropy_dic[L]) == 0:
        entropy_dic.pop(L,None)
        Z_n_entropy_dic.pop(L,None)

L=10_T=50_N=4
L=14_T=70_N=4
L=12_T=60_N=4
L=16_T=80_N=4
L=8_T=40_N=4
L=6_T=30_N=4


## Plot of entropy vs g for fixed p

In [4]:
entropy_dic[12][0].keys()

dict_keys([0.01, 0.2, 0.03, 0.05, 0.0, 0.1])

In [5]:
p_fixed = [0.075]
xdata_dic = {}
ydata_dic = {}
err_dic = {}
full_data = Z_n_entropy_dic
full_data = entropy_dic
for L in sorted(entropy_dic):
    xdata = []
    errdata = []
    ydata = []
    maximum, _ = max_value(L//2-N//2,L,N)
    for p in entropy_dic[L]:
        if p not in p_fixed:
            continue
        print(sorted(entropy_dic[L][p]))
        for g in sorted(entropy_dic[L][p]):
            data = np.average(full_data[L][p][g][:,4*L:4*L+4],axis=1)
            data = np.round(data,10)
            data[data == -np.inf] = 0
            data[data>100] = 0
            data = data
            print(L,p,g,len(data))
            xdata.append(np.average(data))
            errdata.append(np.std(data)/len(data)**0.5)
            ydata.append(g)
    xdata_dic[L] = xdata
    ydata_dic[L] = ydata
    err_dic[L] = errdata

for L in ydata_dic:
    pl.errorbar(ydata_dic[L],xdata_dic[L],yerr = err_dic[L],ls='-',marker='o',label=r'$L=${}'.format(L))
pl.legend()


[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
6 0.075 0.0 1000
6 0.075 0.01 1000
6 0.075 0.03 1000
6 0.075 0.05 1000
6 0.075 0.1 1000
6 0.075 0.2 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
8 0.075 0.0 1000
8 0.075 0.01 1000
8 0.075 0.03 1000
8 0.075 0.05 1000
8 0.075 0.1 1000
8 0.075 0.2 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
10 0.075 0.0 1000
10 0.075 0.01 1000
10 0.075 0.03 1000
10 0.075 0.05 1000
10 0.075 0.1 1000
10 0.075 0.2 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
12 0.075 0.0 1000
12 0.075 0.01 1000
12 0.075 0.03 1000
12 0.075 0.05 1000
12 0.075 0.1 1000
12 0.075 0.2 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
14 0.075 0.0 1000
14 0.075 0.01 1000
14 0.075 0.03 1000
14 0.075 0.05 1000
14 0.075 0.1 1000
14 0.075 0.2 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
16 0.075 0.0 1000
16 0.075 0.01 1000
16 0.075 0.03 1000
16 0.075 0.05 1000
16 0.075 0.1 1000
16 0.075 0.2 1000


## Plot of entropy vs p for fixed g

In [7]:
g_fixed = [0.01]
xdata_dic = {}
ydata_dic = {}
err_dic = {}
full_data = Z_n_entropy_dic
# full_data = entropy_dic
for L in sorted(entropy_dic):
    xdata = {}
    errdata = {}
    ydata = {}
    maximum, _ = max_value(L//2-N//2,L,N)
    maximum = Z_n_max_value([(L//2 - N//2)%N, (L//2)%N],L,N)
    for p in sorted(entropy_dic[L]):
        print(sorted(entropy_dic[L][p]))
        for g in sorted(entropy_dic[L][p]):
            if g not in g_fixed:
                continue
            
            if g not in xdata:
                xdata[g] = []
                ydata[g] = []
                errdata[g] = []

            data = np.average(full_data[L][p][g][:,3*L-L//2:3*L],axis=1)
            data = np.round(data,10)
            data[data == -np.inf] = 0
            data[data>100] = 0
            data = data/np.log(2)

            print(L,p,g,len(data))

            ydata[g].append(np.average(data))
            errdata[g].append(np.std(data)/len(data)**0.5)
            xdata[g].append(p)

    xdata_dic[L] = xdata
    ydata_dic[L] = ydata
    err_dic[L] = errdata

for L in ydata_dic:
    pl.errorbar(xdata_dic[L][g_fixed[0]],ydata_dic[L][g_fixed[0]],yerr = err_dic[L][g_fixed[0]],ls='-',marker='o',label=r'$L=${}'.format(L))
pl.legend()
# pl.yscale('log')

[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
6 0.0 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
6 0.01 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
6 0.03 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
6 0.05 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
6 0.075 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
6 0.085 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
6 0.1 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
6 0.2 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
8 0.0 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
8 0.01 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
8 0.03 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
8 0.05 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
8 0.075 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
8 0.085 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
8 0.1 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
8 0.2 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
10 0.0 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
10 0.01 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
10 0.03 0.01 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.

## Plot vs T for fixed p and g

In [13]:
p_fixed = [0.03]
g_fixed = [0.0]
data_dic = Z_n_entropy_dic
# data_dic = entropy_dic
xdata_dic = {}
ydata_dic = {}
err_dic = {}
alpha_g = (np.linspace(1,0.5,len(g_fixed)))
L_colors = [colorFader('red','green',mix=x) for x in np.linspace(0.2,1,len(entropy_dic))]
for L in sorted(entropy_dic)[:]:
    xdata = {}
    errdata = {}
    ydata = {}
    maximum, _ = max_value(L//2-N//2,L,N)
    for p in sorted(entropy_dic[L]):
        if p not in p_fixed:
            continue
        print(sorted(entropy_dic[L][p]))
        for g in sorted(entropy_dic[L][p]):
            if g not in g_fixed:
                continue
            
            if (p,g) not in xdata:
                xdata[(p,g)] = []
                ydata[(p,g)] = []
                errdata[(p,g)] = []

            data = np.array(data_dic[L][p][g][:,:])
            data = np.round(data,10)
            data[data == -np.inf] = 0
            data[data>100] = 0
            data = data

            print(L,p,g,len(data))

            ydata[(p,g)] = (np.average(data,axis=0))
            errdata[(p,g)] = (np.std(data,axis=0)/len(data)**0.5)
            xdata[(p,g)] = (np.arange(0,5*L+1,1)/L)

    xdata_dic[L] = xdata
    ydata_dic[L] = ydata
    err_dic[L] = errdata


for i,g in enumerate(g_fixed):
    for j,L in enumerate(ydata_dic):
        (p,g) = (p_fixed[0],g)
        pl.errorbar(xdata_dic[L][(p,g)],ydata_dic[L][(p,g)],yerr = err_dic[L][(p,g)],ls='-',marker='o',label=r'$L=${}'.format(L),color=L_colors[j],alpha=alpha_g[i])

pl.legend(ncol=2)
pl.yscale('log')
pl.grid(which='major')

[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
6 0.03 0.0 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
8 0.03 0.0 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
10 0.03 0.0 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
12 0.03 0.0 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
14 0.03 0.0 1000
[0.0, 0.01, 0.03, 0.05, 0.1, 0.2]
16 0.03 0.0 1000


In [176]:
entropy_dic.keys()

dict_keys([])

In [249]:
print('p={},\ng={}'.format(entropy_dic[6].keys(),entropy_dic[6][0].keys()))

p=dict_keys([0.075, 0.01, 0.05, 0.085, 0.2, 0.03, 0.0, 0.1]),
g=dict_keys([0.01, 0.2, 0.03, 0.05, 0.0, 0.1])
